# A. Membuat Model GAN

Langkah pertama yang saya lakukan adalah import beberapa library yang nantinya membantu saya untuk menjalankan pembuatan model GAN nantinya. Lanjutannya, saya melakukan load folder zip yang terdapat image didalamnya.

In [1]:
import zipfile
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm

In [2]:
zip_path = "/content/sample_data/A_23-20250627T052734Z-1-001.zip"
extract_path = "/content/sample_data/images/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

image_size = (100, 100)
add_noise = True

def add_gaussian_noise(image_array, mean=0.0, std=15.0):
    gaussian = np.random.normal(loc=mean, scale=std, size=image_array.shape)
    noisy_img = image_array + gaussian
    return np.clip(noisy_img, 0, 255).astype(np.uint8)

image_files = []
for root, _, files in os.walk(extract_path):
    for filename in files:
        if filename.lower().endswith('.jpg'):
            image_files.append(os.path.join(root, filename))

print(f"Total gambar asli ditemukan: {len(image_files)}")

Total gambar asli ditemukan: 1074


Pada tahap ini saya melakukan split data dengan lanjutan dari soal no 2 sebelumnya.

In [3]:
images, labels = [], []
for file_path in image_files:
    try:
        with Image.open(file_path) as img:
            img = img.convert('RGB')
            img = img.resize(image_size)
            img_arr = np.array(img)
            images.append(img_arr)
            labels.append(os.path.basename(file_path))

            if add_noise:
                noisy_version = add_gaussian_noise(img_arr)
                images.append(noisy_version)
                labels.append(os.path.basename(file_path) + "_noise")
    except Exception as err:
        print(f"Gagal membaca file {file_path}: {err}")

images = np.array(images)
labels = np.array(labels)

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Jumlah total data (termasuk noise): {len(images)}")
print(f"Data latih: {len(X_train)}, Validasi: {len(X_val)}, Uji: {len(X_test)}")

Jumlah total data (termasuk noise): 2148
Data latih: 1718, Validasi: 215, Uji: 215


## i. Generator

Fungsi `build_generator()` membuat model untuk menghasilkan gambar 100×100×3 dari noise berdimensi 100. Model ini terdiri dari tiga layer `Conv2DTranspose` dengan filter 64, 32, dan 3, kernel 3×3, stride 1, padding ‘valid’, dan aktivasi ReLU, kecuali layer terakhir menggunakan aktivasi tanh. Layer awal menggunakan `Dense` untuk memperluas dimensi noise, kemudian diubah bentuknya dengan `Reshape`. Arsitektur ini sesuai ketentuan soal untuk membangun generator dengan output gambar RGB.

In [4]:
def build_generator():
    noise_input = layers.Input(shape=(100,))

    x = layers.Dense(25 * 25 * 64, activation='relu')(noise_input)
    x = layers.Reshape((25, 25, 64))(x)
    x = layers.UpSampling2D(size=(2, 2))(x)  # 50x50

    x = layers.Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    output = layers.Conv2D(3, kernel_size=3, strides=1, padding='same', activation='tanh')(x)

    model = models.Model(inputs=noise_input, outputs=output, name="Generator")
    return model

In [5]:
generator = build_generator()
generator.summary()

Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40000)          │     4,040,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 50, 50, 16)     │         9,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 50, 50, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 100, 100, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 100, 100, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,074,099 (15.54 MB)

 Trainable params: 4,074,099 (15.54 MB)

 Non-trainable params: 0 (0.00 B)

## ii. Diskriminator

Fungsi `build_discriminator()` membuat model untuk mengklasifikasikan gambar 100×100×3 sebagai asli atau palsu. Model terdiri dari tiga layer Conv2D dengan filter 16, 32, dan 64, kernel 3×3, stride 1, padding ‘valid’, dan aktivasi ReLU. Setelah itu, output diratakan dengan `Flatten()` dan diteruskan ke layer dense tunggal dengan aktivasi sigmoid untuk menghasilkan probabilitas keaslian gambar, sesuai arsitektur yang diminta soal.

In [6]:
def build_discriminator():
    image_input = layers.Input(shape=(100, 100, 3))

    x = layers.Conv2D(16, kernel_size=3, strides=1, padding='valid', activation='relu')(image_input)
    x = layers.Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu')(x)
    x = layers.Conv2D(64, kernel_size=3, strides=1, padding='valid', activation='relu')(x)

    x = layers.Flatten()(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=image_input, outputs=output, name="Discriminator")
    return model

In [7]:
discriminator = build_discriminator()
discriminator.summary()

Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 98, 98, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 96, 96, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 94, 94, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 565504)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │       565,505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 589,089 (2.25 MB)

 Trainable params: 589,089 (2.25 MB)

 Non-trainable params: 0 (0.00 B)

## iii. Optimizer Adam & Loss Binary Crossentropy

Tahap ini mengatur pelatihan GAN dengan optimizer Adam (lr=0.0002, beta\_1=0.5) yang umum digunakan agar stabil. Discriminator dikompilasi dan dibekukan agar saat melatih GAN, hanya generator yang belajar. Model `gan_model` menghubungkan noise ke output validitas dari discriminator, lalu dikompilasi dengan loss `binary_crossentropy` untuk melatih generator menghasilkan gambar yang meyakinkan.

In [8]:
gan_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator.compile(optimizer=gan_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

latent_input = layers.Input(shape=(100,))
generated_image = generator(latent_input)
validity_output = discriminator(generated_image)

gan_model = Model(inputs=latent_input, outputs=validity_output, name="GAN")
gan_model.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# B. Modifikasi Model Baseline

Berikutnya saya melakukan modifikasi pada model baseline dengan bertujuan :

Generator_Modified dibuat untuk menghasilkan gambar sintetis dari vektor noise berdimensi 100. Arsitekturnya terdiri dari dense layer yang direshape menjadi fitur map 25×25×128, lalu diperbesar secara bertahap menjadi 100×100×3 melalui Conv2DTranspose. BatchNormalization disisipkan untuk menstabilkan pelatihan. Output-nya berupa gambar RGB dengan aktivasi tanh.

Discriminator_Modified bertugas membedakan gambar asli dan palsu. Input berupa gambar 100×100×3 diproses melalui tiga Conv2D layer dengan Dropout untuk mengurangi overfitting. Setelah itu diflatten dan diklasifikasikan oleh satu Dense unit beraktivasi sigmoid, menghasilkan probabilitas keaslian gambar.

In [9]:
def build_generator_mod():
    latent_vector = layers.Input(shape=(100,))

    y = layers.Dense(25 * 25 * 128, activation='relu')(latent_vector)
    y = layers.Reshape((25, 25, 128))(y)
    y = layers.BatchNormalization()(y)

    y = layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(y)
    y = layers.Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='tanh')(y)

    generator_mod = models.Model(latent_vector, y, name="Generator_Modified")
    return generator_mod

In [10]:
def build_discriminator_mod():
    input_img = layers.Input(shape=(100, 100, 3))

    z = layers.Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu')(input_img)
    z = layers.Dropout(0.25)(z)
    z = layers.Conv2D(64, kernel_size=3, strides=1, padding='valid', activation='relu')(z)
    z = layers.Dropout(0.25)(z)
    z = layers.Conv2D(128, kernel_size=3, strides=1, padding='valid', activation='relu')(z)

    z = layers.Flatten()(z)
    z = layers.Dense(1, activation='sigmoid')(z)

    discriminator_mod = models.Model(input_img, z, name="Discriminator_Modified")
    return discriminator_mod

# C. Evaluasi Model

Tahap ini saya mengevaluasi kualitas gambar yang dihasilkan oleh model generator GAN dengan menghitung nilai Fréchet Inception Distance (FID). Pertama, fungsi calculate_fid() digunakan untuk menghitung jarak statistik antara distribusi fitur gambar nyata dan gambar hasil generator menggunakan model InceptionV3, setelah gambar di-resize ke ukuran 299×299 dan dipreproses. Selanjutnya, 100 sampel noise acak dibuat dan dimasukkan ke generator baseline untuk menghasilkan gambar sintetis, kemudian dibandingkan dengan gambar asli dari X_train untuk mendapatkan nilai FID baseline. Setelah itu, proses yang sama dilakukan menggunakan generator modifikasi untuk menghasilkan gambar dan menghitung FID modifikasi. Kedua nilai FID ini digunakan untuk menilai sejauh mana gambar hasil model mendekati kualitas distribusi gambar asli.

In [11]:
def calculate_fid(real_imgs, gen_imgs):
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

    real_resized = tf.image.resize(real_imgs, (299, 299)).numpy()
    gen_resized = tf.image.resize(gen_imgs, (299, 299)).numpy()

    real_preprocessed = preprocess_input(real_resized)
    gen_preprocessed = preprocess_input(gen_resized)

    real_features = inception_model.predict(real_preprocessed)
    gen_features = inception_model.predict(gen_preprocessed)

    mu_real, sigma_real = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu_gen, sigma_gen = gen_features.mean(axis=0), np.cov(gen_features, rowvar=False)

    diff = np.sum((mu_real - mu_gen) ** 2)
    cov_sqrt = sqrtm(sigma_real @ sigma_gen)

    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real

    fid_value = diff + np.trace(sigma_real + sigma_gen - 2.0 * cov_sqrt)
    return fid_value

In [12]:
sample_count = 100
random_noise = tf.random.normal([sample_count, 100])
gen_images = generator(random_noise, training=False)

real_imgs = X_train[:sample_count].astype('float32') / 255.0
real_resized = tf.image.resize(real_imgs, (299, 299))
gen_resized = tf.image.resize(gen_images, (299, 299))

fid_baseline = calculate_fid(real_resized, gen_resized)
print(f"FID Score Baseline: {fid_baseline:.2f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 8s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 44s 11s/step
FID Score Baseline: 11.10


In [13]:
generator_mod = build_generator_mod()

n_samples = 100
noise_mod = tf.random.normal([n_samples, 100])
generated_images_mod = generator_mod(noise_mod, training=False)

real_images = X_train[:n_samples].astype('float32') / 255.0
real_resized = tf.image.resize(real_images, (299, 299))
generated_resized = tf.image.resize(generated_images_mod, (299, 299))

fid_score_mod = calculate_fid(real_resized, generated_resized)
print(f"FID Score Modifikasi: {fid_score_mod:.2f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 8s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 7s/step
FID Score Modifikasi: 11.51


Hasil evaluasi menunjukkan bahwa skor FID untuk model baseline adalah 11.10, sedangkan model modifikasi mendapatkan skor 11.51. Selisihnya sangat sedikit yang berarti kualitas gambar yang dihasilkan oleh kedua model hampir sama namun model baseline lebih unggul. Oleh karena perbedaan yang tidak signifikan ini, maka model modifikasi masih dapat dianggap berhasil menghasilkan gambar sintetis yang cukup menyerupai data asli. Dengan kata lain, modifikasi yang dilakukan tidak memberikan peningkatan berarti, tetapi juga tidak menurunkan performa secara drastis.

# D. Video Penjelasan

Video terdapat pada link berikut:

https://drive.google.com/drive/folders/1i66HebEyFdobM4UYxXD7Z4uCOQKslo8L?usp=drive_link